Implement Word2Vec algorithm to compute vector representations of words. This example is using a small chunk of Wikipedia articles to train from.

In [1]:
import collections
import os
import random
import urllib
import zipfile

import numpy as np
import tensorflow as tf
os.chdir('D:/data')

C:\Application\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
# Training Parameters
learning_rate = 0.1
batch_size = 128
num_step = 3000000
display_step = 10000
eval_step = 200000

# Evaluation Parameters
eval_words = ['five', 'of', 'going', 'hardware', 'american', 'britain']

# Word2Vec Parameters
embedding_size = 200 # Dimension of the embedding vector
max_vocabulary_size = 50000# Total number of different words in the vocabulary
min_occurrence = 10# Remove all words that does not appears at least n times
skip_window = 3 # How many words to consider left and right
num_skips = 2# How many times to reuse an input to generate a label
num_sampled = 64 # Number of negative examples to sample

In [11]:
# # Download a small chunk of Wikipedia articles collection
# url = 'http://mattmahoney.net/dc/text8.zip'
# data_path = 'text8.zip'

# data_path = 'text8.zip'
# if not os.path.exists(data_path):
#     print("Downloading the dataset... (It may take some time)")
#     filename, _ = urllib.request.urlretrieve(url, data_path)
#     print("Done!")
    
# Unzip the dataset file. Text has already been processed
data_path= 'D:/data/text8.zip'
with zipfile.ZipFile(data_path) as f:
    text_words = f.read(f.namelist()[0]).lower().split()  

In [14]:
# Build the dictionary and replace rare words with UNK token
count = [('UNK',-1)]
# Retrieve the most common words 统计前max_vocabulary_size-1个
count.extend(collections.Counter(text_words).most_common(max_vocabulary_size - 1))
# Remove samples with less than 'min_occurrence' occurrences
for i in range(len(count) - 1, -1, -1):
    if count[i][1] < min_occurrence:
        count.pop(i)
    else:
        # The collection is ordered, so stop when 'min_occurrence' is reached
        break
# Compute the vocabulary size
vocabulary_size = len(count)
# Assign an id to each word
word2id = dict()
for i, (word, _)in enumerate(count):
    word2id[word] = i

data = list()
unk_count = 0
for word in text_words:
    # Retrieve a word id, or assign it index 0 ('UNK') if not in dictionary
    index = word2id.get(word, 0)
    if index == 0:
        unk_count += 1
    data.append(index)
count[0] = ('UNK', unk_count)
id2word = dict(zip(word2id.values(), word2id.keys()))

print("Words count:", len(text_words))
print("Unique words:", len(set(text_words)))
print("Vocabulary size:", vocabulary_size)
print("Most common words:", count[:10])

Words count: 17005207
Unique words: 253854
Vocabulary size: 47135
Most common words: [('UNK', 444176), (b'the', 1061396), (b'of', 593677), (b'and', 416629), (b'one', 411764), (b'in', 372201), (b'a', 325873), (b'to', 316376), (b'zero', 264975), (b'nine', 250430)]
